<a href="https://colab.research.google.com/github/sanjit1995/IdentifyDanceForms_HackerEarth/blob/master/danceForms_HE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Object detection


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/object_detection"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/hub/examples/colab/object_detection.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This Colab demonstrates use of a TF-Hub module trained to perform object detection.

## Setup


In [3]:
#@title Imports and function definitions

# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time

# Print Tensorflow version
print(tf.__version__)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

2.2.0
The following GPU devices are available: /device:GPU:0


## Example use

### Helper functions for downloading images and for visualization.

Visualization code adapted from [TF object detection API](https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py) for the simplest required functionality.

In [0]:
def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)


def download_and_resize_image(url, new_width=256, new_height=256,
                              display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = urlopen(url)
  image_data = response.read()
  image_data = BytesIO(image_data)
  pil_image = Image.open(image_data)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename


def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin

import cv2
from google.colab.patches import cv2_imshow
from IPython.display import display

def draw_boxes(image, boxes, class_names, scores, max_boxes, img_num, min_score=0.2):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()
  
  print(len(boxes))

  start_num = img_num
  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      (frame_height, frame_width) = image.shape[:2]
      #print(np.squeeze(boxes))
      #print(boxes[i][0]*frame_height)
      y_min = int(boxes[i][0]*frame_height)
      x_min = int(boxes[i][1]*frame_width)
      y_max = int(boxes[i][2]*frame_height) 
      x_max = int(boxes[i][3]*frame_width)
      #print(y_max, y_min, x_max, x_min)
      #cropped_img = image[y_max:y_min,x_max:x_min]
      cropped_img = image_pil.crop((x_min, y_min, x_max, y_max))
      #print(image.shape)
      #print(cropped_img)
      #display(cropped_img)
      cropped_img.save("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/test/cropped/" + str(img_num) + ".jpg")
      img_num += 1
      #cv2.imwrite('cropped image_' + str(i) + '.jpg', cropped_img)
      #draw_bounding_box_on_image(image_pil,ymin,xmin,ymax,xmax,color,font,display_str_list=[display_str])
      #np.copyto(image, np.array(image_pil))
    
  if start_num == img_num:
    ymin, xmin, ymax, xmax = tuple(boxes[0])
    display_str = "{}: {}%".format(class_names[0].decode("ascii"),
                                     int(100 * scores[0]))
    color = colors[hash(class_names[0]) % len(colors)]
    image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
    (frame_height, frame_width) = image.shape[:2]
    #print(np.squeeze(boxes))
    #print(boxes[i][0]*frame_height)
    y_min = int(boxes[i][0]*frame_height)
    x_min = int(boxes[i][1]*frame_width)
    y_max = int(boxes[i][2]*frame_height) 
    x_max = int(boxes[i][3]*frame_width)
    #print(y_max, y_min, x_max, x_min)
    #cropped_img = image[y_max:y_min,x_max:x_min]
    cropped_img = image_pil.crop((x_min, y_min, x_max, y_max))
    #print(image.shape)
    #print(cropped_img)
    #display(cropped_img)
    cropped_img.save("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/test/cropped/" + str(img_num) + ".jpg")
    img_num += 1

  return img_num

## Apply module

Load a public image from Open Images v4, save locally, and display.

In [0]:
# By Heiko Gorski, Source: https://commons.wikimedia.org/wiki/File:Naxos_Taverna.jpg
image_url = "https://upload.wikimedia.org/wikipedia/commons/6/60/Naxos_Taverna.jpg"  #@param
downloaded_image_path = download_and_resize_image(image_url, 1280, 856, True)

Pick an object detection module and apply on the downloaded image. Modules:
* **FasterRCNN+InceptionResNet V2**: high accuracy,
* **ssd+mobilenet V2**: small and fast.

In [6]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

detector = hub.load(module_handle).signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

In [0]:
import numpy as np
def run_detector(detector, path, img_num):
  img = load_img(path)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()}

  print("Found %d objects." % len(result["detection_scores"]))
  print("Inference time: ", end_time-start_time)

  #print(result['detection_class_entities'].type())
  #print(result["detection_class_entities"])
  print(len(result["detection_scores"]), len(result["detection_class_entities"]))
  
  detected_classes = result["detection_class_entities"].copy()
  detected_classes = detected_classes[detected_classes == bytes("Person", 'utf-8')]
  
  detected_scores = result["detection_scores"].copy()
  detected_scores = detected_scores[result["detection_class_entities"] == bytes("Person", 'utf-8')]
  
  detected_boxes = result["detection_boxes"].copy()
  detected_boxes = detected_boxes[result["detection_class_entities"] == bytes("Person", 'utf-8')]
  
  detected_scores_temp = detected_scores.copy()
  detected_scores_temp = detected_scores_temp[detected_scores > 0.2]
  #print(detected_scores)
  
  detected_classes_temp = detected_classes.copy()
  detected_classes_temp = detected_classes_temp[detected_scores > 0.2]
  #print(detected_classes)
  
  detected_boxes_temp = detected_boxes.copy()
  detected_boxes_temp = detected_boxes_temp[detected_scores > 0.2]
  #print(detected_boxes)
  #print(result["detection_class_entities"])
  #print(result["detection_scores"])
  #print(len(detected_classes))
  return draw_boxes(
      img.numpy(), detected_boxes,
      detected_classes, detected_scores, max_boxes=len(detected_classes), img_num=img_num)

  #display_image(image_with_boxes)

In [0]:
run_detector(detector, '/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/train/136.jpg', 810)

In [0]:
import os
import pandas as pd

temp_train_labels = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/test.csv")
#print(train_labels.loc[train_labels['Image'] == "96.jpg"])
count=0
#train_labels = pd.DataFrame(columns=['Image','target','parent_image'])
test_labels = pd.DataFrame(columns=['Image','parent_image'])
#print(train_labels)
for filename in os.listdir("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/test/"):
  if ".jpg" in filename:
    print(filename)
    newCount = run_detector(detector, '/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/test/' + filename, count)
    for i in range(count, newCount):
      #train_labels.loc[i] = [str(i) + ".jpg", temp_train_labels.loc[temp_train_labels.Image == str(filename),"target"].values[0], str(filename)]
      test_labels.loc[i] = [str(i) + ".jpg", str(filename)]
    count = newCount
    print("Total files created = " + str(newCount))  
  else:
    continue
#train_labels.to_csv("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/new_train.csv")
test_labels.to_csv("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/new_test.csv")

In [0]:
augmented_test_images

In [0]:
import pandas as pd
train_labels = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/new_train.csv")
del train_labels['Unnamed: 0']
sample_t = train_labels.drop(['Image'], axis=1)
print(train_labels.parent_image.nunique())
temp_train_labels = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/train.csv")
print(temp_train_labels.Image.nunique())
idx1 = pd.Index(train_labels.parent_image)
idx2 = pd.Index(temp_train_labels.Image)

idx1.difference(idx2).values
#sample_t.drop_duplicates()

349
364


array([], dtype=object)

In [0]:
print(train_labels.parent_image.nunique())
print(temp_train_labels.Image.nunique())

364
364


In [0]:
train_labels['target'].value_counts()

kathakali        106
odissi            98
manipuri          82
kuchipudi         79
bharatanatyam     76
mohiniyattam      71
sattriya          68
kathak            66
Name: target, dtype: int64

In [0]:
import cv2

img = cv2.imread('/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/train/cropped/1388.jpg',0)
img.shape

In [0]:
temp_train_labels[~temp_train_labels.Image.isin(train_labels.parent_image)]

In [0]:
#temp_train_labels[temp_train_labels['Image'] == "228.jpg"]
temp_train_labels.loc[temp_train_labels.Image == "134.jpg","target"].values[0]

Augment the images

In [0]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
def augmentData(img_file, noOfNewFiles, saveDir, imageName):
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

    img = load_img(img_file)
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)

    if not os.path.exists(saveDir):
      os.makedirs(saveDir)

    i = 0
    for batch in datagen.flow(x, batch_size=1, save_to_dir=saveDir, save_prefix=imageName, save_format='jpg'):
        i += 1
        if i >= noOfNewFiles:
            break

In [0]:
count = 0
import cv2
for filename in train_labels['Image']:
  print(filename)
  img = "/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/train/cropped/" + str(filename)
  saveDir = "/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/train/augmented/" + str(count)
  augmentData(img, 10, saveDir, count)
  count += 1

Start here for Test Data

In [0]:
test_labels = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/new_test.csv")

resizing and storing test images

In [0]:
import cv2
import os
from google.colab.patches import cv2_imshow
all_test_images = []
for filename in test_labels['Image']:
  #print(filename)
  img = cv2.imread(os.path.join("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/test/cropped/", filename), 0)
  #cv2_imshow(img)
  img = cv2.resize(img, (28, 28))
  if img is not None:
    all_test_images.append(img)
  print("Data loaded for " + str(filename))

In [0]:
all_test_images = np.array(all_test_images)
all_test_images = all_test_images / 255.0

In [0]:
all_test_images

In [0]:
x_test = all_test_images.reshape(len(all_test_images), 28, 28, 1)

In [0]:
x_test.shape

Start from here for Training Data

In [0]:
import pandas as pd
train_labels = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/new_train.csv")
train_labels = train_labels.drop(['Unnamed: 0'], axis=1)

In [0]:
train_labels

In [0]:
train_labels_final = train_labels['target']
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
categorical_train_labels = label_encoder.fit_transform(train_labels_final)
le = LabelEncoder()
le.fit(train_labels['target'])
y_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

Load the augmented data into empty lists

In [0]:
import cv2
import os
from google.colab.patches import cv2_imshow
augmented_train_images = []
augmented_train_labels = []
k = 0
for folder in os.listdir("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/train/augmented"):
  for filename in os.listdir(os.path.join("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/train/augmented/") + str(folder)):
      #print(filename)
      img = cv2.imread(os.path.join("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/train/augmented/" + str(folder), filename), 0)
      #cv2_imshow(img)
      img = cv2.resize(img, (28, 28))
      if img is not None:
        augmented_train_images.append(img)
        #print(train_labels[k])
        augmented_train_labels.append(categorical_train_labels[k])
  print("Data loaded for " + str(folder))
  k += 1
  if not os.path.exists("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/train/augmented/{}".format(k)):
        break

In [26]:
augmented_train_images.shape

(6458, 28, 28)

In [0]:
augmented_train_labels.shape

In [0]:
import numpy as np
# Convert the lists to numpy arrays for further processing and do scaling
augmented_train_images = np.array(augmented_train_images)
augmented_train_images = augmented_train_images / 255.0

In [0]:
y_train = np.array(augmented_train_labels)

In [0]:
# Reshape the data to fit into the model
x_train = augmented_train_images.reshape(len(augmented_train_images), 28, 28, 1)

In [0]:
y_mapping

In [31]:
x_train.shape

(6458, 28, 28, 1)

In [32]:
y_train.shape

(6458,)

In [33]:
x_test.shape

(257, 28, 28, 1)

In [0]:
train_labels

go to training part now

Extra (maybe for later)

In [0]:
import os
def resizeImages():
    for filename in os.listdir("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/train/cropped/"):
        img = cv2.imread(os.path.join("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/train/cropped/", filename), 0)
        img = cv2.resize(img, (32, 32))
        cv2.imwrite("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/train/resized/" + filename, img)

resizeImages()

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Store Images in array

In [0]:
temp_train_images = []
for filename in os.listdir("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/train/resized/"):
  #count += 1
  img = cv2.imread(os.path.join("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/train/resized/", filename), 0)
  img = cv2.resize(img, (28, 28))
  if img is not None:
    temp_train_images.append(img)

train_images = np.array(temp_train_images)
train_images = train_images / 255.0
x_train = train_images.reshape(len(train_images), 28, 28, 1)

In [0]:
run_detector(detector, downloaded_image_path)

Training Part

In [0]:
pip install keras-tuner

In [0]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
        activation='relu',
        input_shape=(28, 28, 1)
    ))
    model.add(keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
        activation='relu'
    ))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ))
    model.add(keras.layers.Dense(8, activation='softmax'))

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model


tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='output',
    project_name='models'
)

tuner.search(x_train, y_train, epochs=3, validation_split=0.1)

model = tuner.get_best_models(num_models=1)[0]

model.summary()

In [0]:
model.fit(x_train, y_train, batch_size=100 , epochs=50, validation_split=0.1, initial_epoch=3)

In [0]:
y_pred = np.argmax(model.predict(x_test), axis=-1)

In [0]:
y_pred

In [0]:
len(y_pred)

append to file

In [0]:
pred = pd.DataFrame(y_pred)
sub_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/new_test.csv")
datasets = pd.concat([sub_df[['Image','parent_image']],pred], axis=1)
datasets.columns = ['Image','parent_image','target']
datasets.target = datasets.target.astype(int)
datasets.to_csv('/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/my_submission1.csv', index=False)

In [0]:
datasets = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/my_submission1.csv')

Find mode of parent_image

In [0]:
new_df = datasets.drop(['Image'], axis=1)
new_df

In [0]:
from scipy.stats import mode
modes = new_df.groupby('parent_image')['target'].apply(lambda x: mode(x)[0][0]).reset_index()

In [0]:
modes.columns = ['Image','target']
modes

In [0]:
test_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/test.csv")

In [0]:
final_df = test_df.merge(modes, on="Image", how = 'inner')

In [0]:
final_df["target"] = pd.to_numeric(final_df["target"])

In [0]:
final_df

In [0]:
y_mapping

In [0]:
target_mapper = dict((v,k) for k,v in y_mapping.items())
target_mapper

In [0]:
df_final = final_df.replace({"target": target_mapper})

In [0]:
df_final

In [0]:
df_final.to_csv('/content/drive/My Drive/Colab Notebooks/data/DanceForms_HE/my_submission_full.csv', index=False)

In [0]:
image_urls = [
  # Source: https://commons.wikimedia.org/wiki/File:The_Coleoptera_of_the_British_islands_(Plate_125)_(8592917784).jpg
  "https://upload.wikimedia.org/wikipedia/commons/1/1b/The_Coleoptera_of_the_British_islands_%28Plate_125%29_%288592917784%29.jpg",
  # By Américo Toledano, Source: https://commons.wikimedia.org/wiki/File:Biblioteca_Maim%C3%B3nides,_Campus_Universitario_de_Rabanales_007.jpg
  "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg/1024px-Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg",
  # Source: https://commons.wikimedia.org/wiki/File:The_smaller_British_birds_(8053836633).jpg
  "https://upload.wikimedia.org/wikipedia/commons/0/09/The_smaller_British_birds_%288053836633%29.jpg",
  ]

def detect_img(image_url):
  start_time = time.time()
  image_path = download_and_resize_image(image_url, 640, 480)
  run_detector(detector, image_path)
  end_time = time.time()
  print("Inference time:",start_time-end_time)

In [0]:
detect_img(image_urls[0])

In [0]:
detect_img(image_urls[1])

In [0]:
detect_img(image_urls[2])